In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()
scaler = StandardScaler()

In [3]:
df = pd.read_csv('~/data/BATADAL_dataset03.csv')
# df = df.iloc[:-1 , :]
df_test = pd.read_csv('~/data/Batadal_test.csv')
# df_test = df_test.iloc[:-19,:]
sensors = [col for col in df.columns if col not in ['DATETIME','ATT_FLAG']]

In [4]:
X_train = pd.DataFrame(index=df.index, columns=sensors, data=scaler.fit_transform(df[sensors]))
Y_train = df.loc[:,'ATT_FLAG']
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [5]:
labels=hankel.fit(np.array(Y_test),1,0.5)
isatt=np.any(labels>0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([ 8,  9, 10, 11, 12, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 39, 40,
       41, 42, 43, 44, 51, 52, 53, 54, 55, 63, 64, 65])

In [6]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    # print(indices[idx])
    return radiis[indices[idx]]

In [7]:
for sens in sensors:
    X = X_train.loc[:,sens].values
    X = hankel.fit(X,1,0.5)
    X=X.T 
    kmeans,optimal_k = cluster.fit(X)
    # optimal_k = 1
    # kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
    kmeans.fit(X)
    # print(optimal_k)
    radiis_normal = []
    weights=[]
    centers=[]
    clusters_R=[]
    clusters_V=[]
    threshold_clusters = np.zeros(optimal_k)
    for i in range(optimal_k):
        cluster_ = X[np.where(kmeans.labels_ == i)[0]]
        r = rank.fit(cluster_)
        clusters_R.append(r)
        # print("r: "+str(r))
        U,Sigma,VT = np.linalg.svd(cluster_)
        V = VT.T
        clusters_V.append(V)
        cluster_ = np.matmul(cluster_,V[:,:r])
        weight,center = meepc.fit(cluster_)
        weights.append(weight)
        centers.append(center)
        var1=np.square(cluster_-center)
        var2=np.matmul(weight,var1.T)
        threshold_clusters[i] = np.max(np.sqrt(var2))
        radiis_normal.append(np.sqrt(var2))
        
   
    Xtest = X_test.loc[:,sens].values
    Xtest=hankel.fit(Xtest,1,0.5)  #size -> 1800x274
    Xtest=Xtest.T  
    distances=np.zeros(optimal_k)
    for i in range(optimal_k):
        Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
        var1=np.square(Xtest_cluster-centers[i])
        var2=np.matmul(weights[i],var1.T)
        distance=np.sqrt(var2)
        distances[i] = np.max(distance)

    # stacked_arr=np.vstack(distances)
    # distances_arr=np.transpose(stacked_arr)
    check_anamoly=threshold_clusters-distances     # positive = inside cluster | negative = outside cluster
    #check for if any datapoint is coming inside the cluster
    is_positive=np.any(check_anamoly>0)
    #indices of those rows
    positive_indices=np.where(is_positive)
    # for i in [[267, 268, 269, 272, 273]]:
    #     print(distances_arr[i])
    for i in range(len(Xtest)):
        if i not in positive_indices:
            print(i,end=' ')
    print("\nnext")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
next
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
next
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
next
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
next
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
next
